In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from datetime import datetime

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

In [3]:
data_one_side = []
data_stat = []
data_rot = []

In [4]:
header = "now,intensity,humidity,temperature,temperature_from_humidity,temperature_from_pressure,pressure,orientation_roll,orientation_pitch,orientation_yaw,compass,compass_raw_x,compass_raw_y,compass_raw_z,gyro_roll,gyro_pitch,gyro_yaw,gyro_raw_x,gyro_raw_y,gyro_raw_z,accelerometer_raw_x,accelerometer_raw_y,accelerometer_raw_z,accelerometer_roll,accelerometer_pitch,accelerometer_yaw\n"

def append_headers(i_start, i_end):
    for i in range(i_start, i_end):
        if i < 10:
            f = open(f"data0{i}.csv", "r+")
        else:
            f = open(f"data{i}.csv", "r+")
        content = f.read()
        f.seek(0, 0)
        f.write(header + content)
        f.close()


In [5]:
def load_data(path):
    for i in range(1, num_runs+1):
        if i < 10:
            filename = f"data0{i}.csv"
        else:
            filename = f"data{i}.csv"

        data_rot.append(pd.read_csv(path+filename).dropna())


In [6]:
def calculate_time(data, i):
    data[i]["now"] = pd.to_datetime(data[i]["now"])
    data[i]["time"] = (data[i]['now'] - data[i]['now'][0]).dt.total_seconds()

In [7]:
def calculate_omega(data, i):
    if "time" not in data[i]:
        calculate_time(data, i)

    data[i]['accelerometer_yaw'] = data[i]['accelerometer_yaw'].apply(np.deg2rad)
    data[i]["accelerometer_yaw"] = np.unwrap(data[i]["accelerometer_yaw"])
    data[i]["omega"] = data[i]["accelerometer_yaw"].diff() / data[i]["time"].diff()

def calculate_omega2(data, i):
    if "time" not in data[i]:
        calculate_time(data, i)

    data[i]["yaw_rad"] = data[i]["gyro_yaw"].apply(np.deg2rad)
    data[i]["yaw_rad"] = np.unwrap(data[i]["yaw_rad"])
    data[i]["omega"] = data[i]["yaw_rad"].diff() / data[i]["time"].diff()

def calculate_omega3(data, i):
    if "time" not in data[i]:
        calculate_time(data, i)

    data[i]["omega"] = data[i]["gyro_raw_z"]


In [8]:
def plot_omega(data, i):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    fig.suptitle("Graphs of rotational speed as a function of time", fontsize=24)

    # ax.set_ylim([-8, 8])
    ax.scatter(data[i]["time"], data[i]["gyro_raw_z"])
    ax.title.set_text("")
    ax.set_xlabel("time [s]")
    ax.set_ylabel("Rotational speed [rad / s]")
    plt.show()

In [9]:
def calculate_omegas(data):
    for i in range(len(data)):
        calculate_omega(data, i)

In [10]:
def plot_intensity(data, i):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    fig.suptitle("Graphs of intensity as a function of time", fontsize=24)

    # ax.set_ylim([0.8, 0.92])
    ax.scatter(data[i]["time"], data[i]["intensity"])
    ax.title.set_text("")
    ax.set_xlabel("time [s]")
    ax.set_ylabel("Intensity [%]")
    plt.show()

In [11]:
def plot_intensity_vs_omega(data, i):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    fig.suptitle("Graphs of intensity as a function of omega", fontsize=24)

    # ax.set_ylim([0.8, 0.92])
    ax.scatter(abs(data[i]["gyro_raw_z"]), data[i]["intensity"])
    ax.title.set_text("")
    ax.set_xlabel("omega [rad / s]")
    ax.set_ylabel("Intensity [%]")
    plt.show()

In [12]:
num_runs = 23
load_data("")


FileNotFoundError: [Errno 2] No such file or directory: 'data01.csv'

In [ ]:
i = num_runs-1

In [ ]:
# for i in range(num_runs-1, num_runs):
calculate_time(data_rot, i)
plot_omega(data_rot, i)
plot_intensity(data_rot, i)
plot_intensity_vs_omega(data_rot, i)